# QAS Coding Using LLM (API Version)

### Set file path, api key, model, and prompt version

In [ ]:
input_file_path = "Copy of GSS QAS LLM RCoding 2024.xlsx" # "testing.xlsx"
sheet_name = "All GSS items for LLM"
QAS_pdf_file_path = "QAS-99 Manual.pdf"

model = "gpt-4o-mini" # "gpt-4o" or "gpt-4o-mini"
prompt_version = 3 # 1, 2, or 3

api_key = ""

### Self define functions

In [ ]:
import re
import os
import time
import pandas as pd
from openai import OpenAI
from openpyxl import load_workbook
from openpyxl.styles import Border, Side
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

with open('prompts/prompt_v1.txt', 'r', encoding='utf-8') as file:
    prompt_v1 = file.read()

with open('prompts/prompt_v2.txt', 'r', encoding='utf-8') as file:
    prompt_v2 = file.read()

with open('prompts/prompt_v3.txt', 'r', encoding='utf-8') as file:
    prompt_v3 = file.read()

if prompt_version == 1:
    prompt = prompt_v1
elif prompt_version == 2:
    prompt = prompt_v2
elif prompt_version == 3:
    prompt = prompt_v3

def create_a_thread(client, prompt, QAS_pdf_file_path):
    if prompt_version == 1 or prompt_version == 2:
        thread = client.beta.threads.create(
            messages=[
            {
                "role": "user",
                "content": prompt
            }
            ]
        )
    elif prompt_version == 3:
        message_file = client.files.create(
            file=open(QAS_pdf_file_path, "rb"), purpose="assistants"
        )

        thread = client.beta.threads.create(
            messages=[
            {
                "role": "user",
                "content": prompt,
                # Attach the new file to the message.
                "attachments": [{"file_id": message_file.id, "tools": [{"type": "file_search"}]}],
            }
            ]
        )
        # print(thread.tool_resources.file_search)
    
    return thread


def run_a_message(client, thread_id, assistant_id, content):
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

    run = client.beta.threads.runs.create(thread_id=thread_id, assistant_id=assistant_id, instructions="Please follow the QAS and provide an answer in the intended format.")

    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        time.sleep(1)

    message_response = client.beta.threads.messages.list(thread_id=thread_id)
    message_response = message_response.data
    return message_response[0].content[0].text.value


def process_response(response):
    # 1. Extract variables and values using regular expressions
    variable_match = re.findall(r'(\d+[a-z])\. (\d+)', response)
    coding_results = {variable: int(value) for variable, value in variable_match}
    QAS_issues = ['1a', '1b', '1c', '2a', '2b', '3a', '3b', '3c', '3d', '4a', '4b', '4c', '5a', '5b', '5c', '5d', '6a', '6b', '6c', '7a', '7b', '7c', '7d', '7e', '7f', '7g', '8a']
    for issue in QAS_issues:
        if issue not in coding_results:
            coding_results[issue] = 0
    coding_results = dict(sorted(coding_results.items()))

    # 2. Calculate the sum
    all_sum = sum(coding_results.values())

    category_totals = {}
    for key, value in coding_results.items():
        main_category = key[0]  # Extract the main category as the numeric part of the key (e.g., '1' from '1a')

        if main_category in category_totals:
            category_totals[main_category] += value
        else:
            category_totals[main_category] = value

    # 3. Extract the explanation
    explanation_match = re.search(r'Explanations?:\s*(.*)', response)
    if explanation_match:
        explanation = explanation_match.group(1)
    else:
        explanation = ""
    
    # 4. Create a DataFrame with a single row for the wide format
    combined_results = pd.DataFrame({**coding_results, 'Explanations': explanation, 'Sum-All': all_sum}, index=[0])
    
    for category, category_sum in category_totals.items():
        combined_results[f'Sum-{category}'] = category_sum
    
    return combined_results


def save_result_to_file(df, output_file_path):
    if os.path.exists(output_file_path):
        # If file exists, append to it
        writer = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay')
        startrow = list(writer.sheets.values())[0].max_row
        df.to_excel(writer, index=False, header=False, startrow=startrow)
        writer.close()
    else:
        # If file doesn't exist, create a new one and format it
        df.to_excel(output_file_path, index=False)

    wb = load_workbook(output_file_path)
    ws = wb.active

    no_border = Border(left=Side(border_style=None),right=Side(border_style=None), top=Side(border_style=None), bottom=Side(border_style=None))
    for row in ws.iter_rows():
        for cell in row:
            cell.border = no_border
            cell.alignment = Alignment(horizontal="center", vertical="center")

    for row in range(2, ws.max_row + 1):
        ws.row_dimensions[row].height = 120

    for col in ws.iter_cols(min_col=1, max_col=ws.max_column):
        col_name = col[0].value
        col_letter = get_column_letter(col[0].column)
        if col_name in ['Variable Name', 'LLM Model', 'Prompt Version', 'Testing Time']:
            ws.column_dimensions[col_letter].width = 15
        elif col_name in ['Full Question Wording', 'Explanations']:
            ws.column_dimensions[col_letter].width = 50
            for cell in col:
                cell.alignment = Alignment(horizontal="left", vertical="top", wrap_text=True)
        elif col_name == 'LLM Coding Results':
            ws.column_dimensions[col_letter].width = 20
            for cell in col:
                cell.alignment = Alignment(horizontal="left", vertical="center", wrap_text=False)
        else:
            ws.column_dimensions[col_letter].width = 8

    for cell in ws["A1:AQ1"][0]:  # Header formatting
        cell.font = Font(bold=True)
        cell.fill = PatternFill("solid", fgColor="00C0C0C0")
        cell.alignment = Alignment(horizontal="center", vertical="center")

    for row in range(1, ws.max_row+1): 
        ws[f'B{row}'].border = Border(right=Side(style="thin"))
        ws[f'F{row}'].border = Border(right=Side(style="thin"))
        ws[f'AH{row}'].border = Border(right=Side(style="thin"))

    wb.save(output_file_path)

def save_result_to_existing_file(df, output_file_path):
    writer = pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay')
    startrow = list(writer.sheets.values())[0].max_row
    df.to_excel(writer, index=False, header=False, startrow=startrow)
    writer.close()

def clean_up(client): # clear up all the assistants, vector stores, and files
    for assistant in client.beta.assistants.list().data:
        client.beta.assistants.delete(assistant.id)
    for vector_store in client.beta.vector_stores.list().data:
        client.beta.vector_stores.delete(vector_store.id)
    for file in client.files.list().data:
        client.files.delete(file.id)
    return len(client.beta.assistants.list().data), len(client.beta.vector_stores.list().data), len(client.files.list().data)

### Read the excel file as input

In [ ]:
df_input = pd.read_excel(input_file_path, engine='openpyxl', sheet_name=sheet_name)
df_input.loc[df_input['Variable Name'].isna(), 'Variable Name'] = 'none' # Fill missing variable names with 'none'
df_input = df_input.loc[~df_input['Variable Name'].str.contains('Notes')] # Remove rows whos variable name contains 'Notes'
df_input['Combined'] = df_input['Variable Name'] + ': ' + df_input['Full Question Wording']
df_input = df_input.dropna(subset=['Full Question Wording'])
df_input = df_input.reset_index(drop=True)

print("Number of rows in df_input: ", len(df_input))

# Only run the questions in m to n-1 rows!
df_output = df_input.iloc[120:].copy()

df_output[:10]

- [x] ~60
- [x] 60~120
- [ ] 120~180
- [ ] 180~

### Run the main coding process

[🔗 Check OpenAI API usage](https://platform.openai.com/usage)

In [ ]:
# 1. Create an assistant
client = OpenAI(api_key=api_key)

if prompt_version == 1 or prompt_version == 2:
    assistant = client.beta.assistants.create(
        name="QAS Coder",
        instructions=prompt,
        model=model,
    )
    vector_store_id = None
elif prompt_version == 3:
    # Step 1: Create a new Assistant with File Search Enabled
    assistant = client.beta.assistants.create(
        name="QAS Coder",
        instructions=prompt,
        model=model,
        tools=[{"type": "file_search"}]
    )
    # Step 2: Upload files and add them to a Vector Store
    vector_store = client.beta.vector_stores.create(name="QAS PDF")
    vector_store_id = vector_store.id
    file_paths = [QAS_pdf_file_path]
    file_streams = [open(path, "rb") for path in file_paths]
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, files=file_streams
    )
    # Step 3: Update the assistant to use the new Vector Store
    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )
    # print(file_batch.status)
    # print(file_batch.file_counts)

# 2. Create a new thread
thread = create_a_thread(client, prompt, QAS_pdf_file_path)

# 3. Process each question and store the results in the DataFrame
print(f"The number of questions to be coded: {len(df_output)}\nStarting the coding process...", end='')
for index, row in df_output.iterrows():
    print(index+1, end='.')
    question = row['Combined']

    # every 20 questions, create a new thread (i.e., start a new conversation) 
    if index % 20 == 19:
        thread = create_a_thread(client, prompt, QAS_pdf_file_path)
        print('\n', end='')

    # send the question to the thread and get the response
    response = run_a_message(client, thread.id, assistant.id, question)
    print('..', end='')

    df_output.at[index, 'LLM Model'] = model
    df_output.at[index, 'Prompt Version'] = prompt_version
    df_output.at[index, 'Testing Time'] = time.strftime("%Y-%m-%d %H:%M")
    df_output.at[index, 'LLM Coding Results'] = response

    # put the processed reponse after the end of this row 
    processed_response = process_response(response)
    for col in processed_response.columns:
        df_output.at[index, col] = processed_response[col].values

df_output.drop(columns=['Combined'], inplace=True)

# 4. Save the results to a new Excel file
output_file_path = input_file_path.replace(".xlsx", "_results.xlsx")

save_result_to_file(df_output, output_file_path)
print(f' "{output_file_path}" Saved! It contains the coding results for {pd.read_excel(output_file_path).shape[0]} questions in total.')
clean_up(client)

For every 60 survey questions, it takes how many minutes and how much money:
-  **gpt-4o-mini + prompt version 1**: 13m; 0.06 USD
-  **gpt-4o-mini + prompt version 2**: 11m; 0.12 USD 
-  **gpt-4o-mini + prompt version 3**: 18m; 0.25 USD